# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [13]:
# your code here

import pandas as pd
import numpy as np

tic = pd.read_csv("tic-tac-toe.csv")

print(tic.head())
print(tic.shape)
print(tic.isnull().sum())


  TL TM TR ML MM MR BL BM BR  class
0  x  x  x  x  o  o  x  o  o   True
1  x  x  x  x  o  o  o  x  o   True
2  x  x  x  x  o  o  o  o  x   True
3  x  x  x  x  o  o  o  b  b   True
4  x  x  x  x  o  o  b  o  b   True
(958, 10)
TL       0
TM       0
TR       0
ML       0
MM       0
MR       0
BL       0
BM       0
BR       0
class    0
dtype: int64


In [14]:
#Split into Predictors and target

# Need to Encode both the Predictor and target using One-Hot Encoding. 

tic_predictors = tic.drop(columns = ["class"])
tic_pred_encoded = tic_predictors.replace({'x': 1, 'o': 0, 'b': 2})

tic_tgt_encoded = tic['class'].astype(int)

print(tic_predictors.head())
print(tic_tgt_encoded.head())
print(tic_pred_encoded.head())


  TL TM TR ML MM MR BL BM BR
0  x  x  x  x  o  o  x  o  o
1  x  x  x  x  o  o  o  x  o
2  x  x  x  x  o  o  o  o  x
3  x  x  x  x  o  o  o  b  b
4  x  x  x  x  o  o  b  o  b
0    1
1    1
2    1
3    1
4    1
Name: class, dtype: int32
   TL  TM  TR  ML  MM  MR  BL  BM  BR
0   1   1   1   1   0   0   1   0   0
1   1   1   1   1   0   0   0   1   0
2   1   1   1   1   0   0   0   0   1
3   1   1   1   1   0   0   0   2   2
4   1   1   1   1   0   0   2   0   2


C:\Users\KK\AppData\Local\Temp\ipykernel_14608\2294999656.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tic_pred_encoded = tic_predictors.replace({'x': 1, 'o': 0, 'b': 2})


In [15]:
n_cols = tic_predictors.shape[1]
print(n_cols)

9


## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [16]:
# your code here

#!pip install tensorflow
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense


In [17]:
from sklearn.model_selection import train_test_split

# Split the Train and Test Data
X_train, X_test, y_train, y_test = train_test_split(tic_pred_encoded,tic_tgt_encoded,test_size=0.2, random_state = 42)

In [6]:
# Define a Sequential Model
def sequential_model():
    #create model
    model = Sequential()
    model.add(Dense(64, activation= 'relu', input_shape=(n_cols,)))
    model.add(Dense(32, activation= 'relu'))
    model.add(Dense(16, activation= 'relu'))
    model.add(Dense(2, activation='softmax')) # Output Layer 

    #compile model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])
    return model

In [7]:
# Build the Model
model = sequential_model()

c:\Users\KK\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Fit the Model
model.fit(X_train, y_train, epochs=50, verbose=2)

Epoch 1/50
24/24 - 4s - 156ms/step - accuracy: 0.6540 - loss: 0.6462
Epoch 2/50
24/24 - 0s - 8ms/step - accuracy: 0.6540 - loss: 0.6143
Epoch 3/50
24/24 - 0s - 8ms/step - accuracy: 0.6580 - loss: 0.5911
Epoch 4/50
24/24 - 0s - 6ms/step - accuracy: 0.6815 - loss: 0.5689
Epoch 5/50
24/24 - 0s - 9ms/step - accuracy: 0.6945 - loss: 0.5459
Epoch 6/50
24/24 - 0s - 6ms/step - accuracy: 0.7337 - loss: 0.5172
Epoch 7/50
24/24 - 0s - 5ms/step - accuracy: 0.7663 - loss: 0.4839
Epoch 8/50
24/24 - 0s - 4ms/step - accuracy: 0.7663 - loss: 0.4542
Epoch 9/50
24/24 - 0s - 4ms/step - accuracy: 0.8460 - loss: 0.4035
Epoch 10/50
24/24 - 0s - 3ms/step - accuracy: 0.8838 - loss: 0.3632
Epoch 11/50
24/24 - 0s - 4ms/step - accuracy: 0.9099 - loss: 0.3251
Epoch 12/50
24/24 - 0s - 3ms/step - accuracy: 0.9204 - loss: 0.2899
Epoch 13/50
24/24 - 0s - 4ms/step - accuracy: 0.9347 - loss: 0.2533
Epoch 14/50
24/24 - 0s - 4ms/step - accuracy: 0.9569 - loss: 0.2223
Epoch 15/50
24/24 - 0s - 3ms/step - accuracy: 0.9543 - 

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [9]:
# your code here

# Make Predictions
y_pred = model.predict(X_test)

print(type(y_pred))
print(type(y_test))


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
<class 'numpy.ndarray'>
<class 'pandas.core.series.Series'>


In [10]:
""" from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred)) """

' from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score\n\nprint("Accuracy:", accuracy_score(y_test, y_pred))\nprint("Precision:", precision_score(y_test, y_pred))\nprint("Recall:", recall_score(y_test, y_pred))\nprint("F1-Score:", f1_score(y_test, y_pred)) '

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [18]:
# HOW TO IMPROVE THE MODEL!

# Adjusting the LEARNING RATE for the model 

from tensorflow.keras.optimizers import Adam

# Define customer learning rate
learning_rate = 0.01 

#Create an instance of the Adam optimizer with the customer learning rate
learning_optimizer = Adam(learning_rate=learning_rate)

# Define a Sequential Model
def sequential_model():
    #create model
    model = Sequential()
    model.add(Dense(64, activation= 'relu', input_shape=(n_cols,)))
    model.add(Dense(32, activation= 'relu'))
    model.add(Dense(16, activation= 'relu'))
    model.add(Dense(2, activation='softmax')) # Output Layer 

    #compile model
    model.compile(optimizer=learning_optimizer, loss='sparse_categorical_crossentropy', metrics = ['accuracy'])  
    return model


In [19]:
# Build the Model
model = sequential_model()

# Fit the Model
model.fit(X_train, y_train, epochs=50, verbose=2)

# Make Predictions
y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score

y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test, y_pred_classes)
print(f'Accuracy: {accuracy}')

c:\Users\KK\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
24/24 - 2s - 102ms/step - accuracy: 0.6397 - loss: 0.6280
Epoch 2/50
24/24 - 0s - 4ms/step - accuracy: 0.6815 - loss: 0.5824
Epoch 3/50
24/24 - 0s - 3ms/step - accuracy: 0.7193 - loss: 0.5216
Epoch 4/50
24/24 - 0s - 3ms/step - accuracy: 0.8016 - loss: 0.4173
Epoch 5/50
24/24 - 0s - 3ms/step - accuracy: 0.8446 - loss: 0.3280
Epoch 6/50
24/24 - 0s - 3ms/step - accuracy: 0.8916 - loss: 0.2692
Epoch 7/50
24/24 - 0s - 3ms/step - accuracy: 0.9360 - loss: 0.1865
Epoch 8/50
24/24 - 0s - 4ms/step - accuracy: 0.9517 - loss: 0.1330
Epoch 9/50
24/24 - 0s - 4ms/step - accuracy: 0.9595 - loss: 0.0996
Epoch 10/50
24/24 - 0s - 3ms/step - accuracy: 0.9648 - loss: 0.0993
Epoch 11/50
24/24 - 0s - 7ms/step - accuracy: 0.9700 - loss: 0.0822
Epoch 12/50
24/24 - 0s - 2ms/step - accuracy: 0.9817 - loss: 0.0625
Epoch 13/50
24/24 - 0s - 3ms/step - accuracy: 0.9765 - loss: 0.0675
Epoch 14/50
24/24 - 0s - 4ms/step - accuracy: 0.9843 - loss: 0.0495
Epoch 15/50
24/24 - 0s - 7ms/step - accuracy: 0.9856 - 

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here

# 1) Adding additional hidden layer - during Model definition
# 2) Increasing the neurons (Dense) in each layer - during Model definition
# 3) Defining the learning rate - during Model definition
# 4) Increasing the epochs - during Model Fit


# Check the above for Accuracy and Loss

# Having epochs of 25 yields accuracy of 0.9491 and loss of 0.1955 
# Having epochs of 50 yields accuracy of 0.9883 and loss of 0.0484 
# Adding another layer and having epochs of 50 yields accuracy of 0.99873 and loss of 0.0135 /* Accuracy didn't improve any better after this!
# Defining the learning rate (as 0.01) improves the accuracy to 1 and brings down the Loss to almost zero. 